<a href="https://colab.research.google.com/github/qayyumu/GradioExp/blob/main/session_04/Gradio_and_Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Environment

In [ ]:
%%capture
!pip install gradio
!pip install transformers


### Import necessary libraries 

In [ ]:

import gradio as gr
from transformers import pipeline
import numpy as np

### configure model pipeline to be used as backend

In [ ]:

model = pipeline("text-generation")


def predict(prompt):
    completion = model(prompt)[0]["generated_text"]
    return completion

### Launch UI interface wiht desired pipeline

In [ ]:
import gradio as gr

gr.Interface(fn=predict, inputs="text", outputs="text").launch()

### Audio to text Interface

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")


In [ ]:
def transcribe(stream, new_chunk):
    try:
        sr, y = new_chunk
        y = y.astype(np.float32)
        y /= np.max(np.abs(y))

        if stream is not None:
            stream = np.concatenate([stream, y])
        else:
            stream = y
        return stream, transcriber({"sampling_rate": sr, "raw": stream})["text"]
    
    except:
        return stream, "problem detected"



In [ ]:
demo = gr.Interface(
    transcribe,
    ["state", gr.Audio(sources=["microphone"], streaming=False)],
    ["state", "text"],
    live=False,css="footer{display:none !important}",
)


In [ ]:
demo.launch(debug=True)
